# Packages Needed

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Function blocks for ETL

In [ ]:
def train_test_split_by_date(df, split_date="2024-01-01"):
  data_raw = df.copy()
  train = data_raw[data_raw["date"] < split_date]
  test = data_raw[data_raw["date"] >= split_date]
  return train, test

def add_unique_station_count(df, window_size=365):
  data_raw = df.copy()
  all_start_stations = data_raw["start_station_name"].unique()
  all_end_stations = data_raw["end_station_name"].unique()
  all_stations = list(set(all_start_stations) | set(all_end_stations))
  start_station_bool = data_raw.groupby(["date", "start_station_name"])["start_station_name"].count().unstack(fill_value=0) > 0
  end_station_bool = data_raw.groupby(["date", "end_station_name"])["end_station_name"].count().unstack(fill_value=0) > 0


  station_count_bool = start_station_bool | end_station_bool
  station_count_bool.fillna(0,inplace=True)
  station_count_bool = station_count_bool.astype(int)
  rolling_station_count = station_count_bool.rolling(window=365, min_periods=1).sum()
  unique_stations_per_window = rolling_station_count.apply(lambda x: len(x[x > 0]), axis=1)
  station_count_bool[f'unique_stations_{window_size}d'] = unique_stations_per_window
  station_count_bool[f'unique_stations_{window_size}d'] = station_count_bool[f'unique_stations_{window_size}d'].shift(1) # shift because we can't use today's value
  return station_count_bool[[f'unique_stations_{window_size}d']].reset_index()[window_size:]

def add_ebike_proportion(df, window_size=365):
  data_raw = df.copy()
  daily_ebike_count = data_raw.groupby('date')['rideable_type'].apply(lambda x: (x == 'electric_bike').sum())
  daily_ebike_count.rename('ebike_count', inplace=True)
  daily_bike_count = data_raw.groupby('date')['rideable_type'].count()
  daily_bike_count.rename('bike_count', inplace=True)
  daily_bike_count = pd.merge(daily_bike_count, daily_ebike_count, how="left",left_index=True, right_index=True)
  daily_bike_count.head()
  rolling_bike_count = daily_bike_count.rolling(window=window_size, min_periods=1).sum()
  rolling_bike_count['ebike_proportion'] = rolling_bike_count['ebike_count'] / rolling_bike_count['bike_count']
  return rolling_bike_count[['ebike_proportion']].reset_index()[window_size:]

def add_date_features(df):
  data_raw = df.copy()
  data_raw["date"] = pd.to_datetime(data_raw["date"])
  data_raw.sort_values("date",inplace=True)
  data_raw["month"] = data_raw["date"].dt.month
  data_raw["dayofweek"] = data_raw["date"].dt.dayofweek
  data_raw["year"] = data_raw["date"].dt.year
  return data_raw

def add_weather_features(df, weather_path):
  grouped_df = df.copy()
  # load weather
  weather = pd.read_csv(weather_path)
  weather["time"] = pd.to_datetime(weather["time"])
  weather.sort_values("time",inplace=True)
  # merge
  grouped_df = grouped_df.merge(weather[["time","temp_min_c","rain_sum_mm","snowfall_sum_cm"]],
                  left_on="date",
                  right_on="time",
                  how="left"
                  )
  return grouped_df

# Main

In [ ]:
# Data loading
df_all = pd.read_parquet("/content/drive/Shared drives/Time Series/divvy_data/qa/qa_divvy_data.parquet")

# Daily count
grouped_df = df_all.groupby("date")["rides"].sum().reset_index()

# Add station availability
grouped_df = grouped_df.merge(add_unique_station_count(df_all,364), on="date", how="left")

# Add e-bike availability
grouped_df = grouped_df.merge(add_ebike_proportion(df_all,364), on="date", how="left")

# Train Test split
train, test = train_test_split_by_date(grouped_df)

# Extract date features
train = add_date_features(train)
test = add_date_features(test)

# Add weather features
train = add_weather_features(train,
          weather_path="/content/drive/Shared drives/Time Series/weather_data/daily_weather_chicago.csv")
test = add_weather_features(test,
          weather_path="/content/drive/Shared drives/Time Series/weather_data/daily_weather_forecast_chicago.csv" )
train.head()

,date,rides,unique_stations_364d,ebike_proportion,month,dayofweek,year,time,temp_min_c,rain_sum_mm,snowfall_sum_cm
0,2020-01-01,2141,NaN,NaN,1,2,2020,2020-01-01,-6.1,0.0,0.00
1,2020-01-02,6479,NaN,NaN,1,3,2020,2020-01-02,2.5,0.0,0.00
2,2020-01-03,5890,NaN,NaN,1,4,2020,2020-01-03,0.2,0.0,0.00
3,2020-01-04,3187,NaN,NaN,1,5,2020,2020-01-04,-2.5,0.0,1.40
4,2020-01-05,3035,NaN,NaN,1,6,2020,2020-01-05,-4.7,0.0,0.07


In [ ]:
train.tail()

,date,rides,unique_stations_364d,ebike_proportion,month,dayofweek,year,time,temp_min_c,rain_sum_mm,snowfall_sum_cm
1454,2023-12-27,6001,1612.0,0.568238,12,2,2023,2023-12-27,-0.1,0.0,0.00
1455,2023-12-28,3888,1611.0,0.568260,12,3,2023,2023-12-28,2.8,5.0,0.00
1456,2023-12-29,4877,1611.0,0.568264,12,4,2023,2023-12-29,-1.1,3.6,0.00
1457,2023-12-30,5073,1612.0,0.568246,12,5,2023,2023-12-30,-2.9,0.0,0.00
1458,2023-12-31,2991,1612.0,0.568281,12,6,2023,2023-12-31,-1.4,0.0,1.12


In [ ]:
test.head()

,date,rides,unique_stations_364d,ebike_proportion,month,dayofweek,year,time,temp_min_c,rain_sum_mm,snowfall_sum_cm
0,2024-01-01,3643,1611.0,0.568322,1,0,2024,2024-01-01,-1.9,0.2,0.0
1,2024-01-02,6532,1612.0,0.568359,1,1,2024,2024-01-02,-2.8,0.0,0.0
2,2024-01-03,7462,1613.0,0.568451,1,2,2024,2024-01-03,-0.6,0.0,0.0
3,2024-01-04,8114,1614.0,0.568423,1,3,2024,2024-01-04,-1.2,0.0,0.0
4,2024-01-05,7377,1613.0,0.568402,1,4,2024,2024-01-05,-1.8,0.0,0.0


In [ ]:
train.to_csv("/content/drive/Shared drives/Time Series/divvy_data/train_2020_to_2023.csv", index=False)
test.to_csv("/content/drive/Shared drives/Time Series/divvy_data/test_2024_to_2025.csv", index=False)